This notebook is apart of the IBM Data Science Capstone on Coursera. The goal is to segment and cluster different neighborhoods in Toronto. I do this by scraping web data from wikipedia about different boroughs and then access the foursquare API to explore different places within those boroughs for the cluster analysis.

In [110]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq

import pandas as pd

In [111]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [112]:
# open connection and grab page
uClient = ureq(url)

In [113]:
html = uClient.read()


In [114]:
uClient.close()

In [115]:
# parse html
soup = soup(html, "html.parser")

In [116]:
df_dict = {"PostalCode":[], "Borough":[], "Neighborhoods":[]}
for tr in soup.find_all('tr'):
    tds = tr.find_all('td')
    br = False
    for td in tds:
        # make sure we are still in the main table
        if td.b is None or td.b.text[0] != 'M':
            br = True
            break
        # make sure postal code is assigned
        curr_td_code = td.b.text
        if td.span is not None:
            txt = td.span.text
            spl = txt.strip(")").split("(")
            borough = spl[0]
            
            # special case
            if borough == "Queen's Park\n":
                #print("q's park")
                df_dict["PostalCode"].append(td.findAll('b')[0].text)
                df_dict["Borough"].append("Queen's Park")
                df_dict["Neighborhoods"].append([td.findAll('b')[1].text])
            
            elif borough != 'Not assigned':
                df_dict["PostalCode"].append(td.b.text)
                df_dict["Borough"].append(borough)
                
                neighborhoods = []
                # for loop because some cells have multiple paranethesis 
                for i in range(1,len(spl)):
                    hoods = spl[i].replace(")", " ").replace(",", "/").split("/")
                    for hood in hoods:
                        neighborhoods.append(hood.strip())
                df_dict["Neighborhoods"].append(neighborhoods)
    # abort mission once we leave the main table
    if br:
        break

In [117]:
df = pd.DataFrame.from_dict(df_dict)
df = df.reindex(columns=["PostalCode", "Borough", "Neighborhoods"])
df.head()

,PostalCode,Borough,Neighborhoods
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]"
4,M7A,Queen's Park,[Ontario Provincial Government]


In [119]:
df.shape

(103, 3)